In [3]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py


--2024-09-02 07:41:08--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-09-02 07:41:08 (78.0 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [44]:
from helper_functions import create_tensorboard_callback,plot_loss_curves,compare_historys,unzip_data,walk_through_dir,make_confusion_matrix

In [13]:
# Download data from Google Storage (already preformatted)
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip

unzip_data("101_food_classes_10_percent.zip")

train_dir = "101_food_classes_10_percent/train/"
test_dir = "101_food_classes_10_percent/test/"

--2024-09-02 07:46:13--  https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.157.207, 142.251.8.207, 142.251.170.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.157.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1625420029 (1.5G) [application/zip]
Saving to: ‘101_food_classes_10_percent.zip’

101_food_classes_10 100%[===================>]   1.51G  17.4MB/s    in 84s     

2024-09-02 07:47:38 (18.5 MB/s) - ‘101_food_classes_10_percent.zip’ saved [1625420029/1625420029]



In [4]:
walk_through_dir("/content/101_food_classes_10_percent")

There are 2 directories and 0 images in '/content/101_food_classes_10_percent'.
There are 101 directories and 0 images in '/content/101_food_classes_10_percent/test'.
There are 0 directories and 250 images in '/content/101_food_classes_10_percent/test/fried_calamari'.
There are 0 directories and 250 images in '/content/101_food_classes_10_percent/test/spaghetti_bolognese'.
There are 0 directories and 250 images in '/content/101_food_classes_10_percent/test/carrot_cake'.
There are 0 directories and 250 images in '/content/101_food_classes_10_percent/test/pizza'.
There are 0 directories and 250 images in '/content/101_food_classes_10_percent/test/falafel'.
There are 0 directories and 250 images in '/content/101_food_classes_10_percent/test/lasagna'.
There are 0 directories and 250 images in '/content/101_food_classes_10_percent/test/crab_cakes'.
There are 0 directories and 250 images in '/content/101_food_classes_10_percent/test/tiramisu'.
There are 0 directories and 250 images in '/cont

In [14]:
import tensorflow as tf
IMG_SIZE=(224,224)
train_data_all_10_percent=tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                              label_mode='categorical',
                                                                              image_size=IMG_SIZE)

test_data=tf.keras.preprocessing.image_dataset_from_directory(test_dir,label_mode='categorical',
                                                              image_size=IMG_SIZE,shuffle=False) #don't shuffle test for prediction analysis

Found 7575 files belonging to 101 classes.
Found 25250 files belonging to 101 classes.


In [15]:
checkpoint_path = "101_classes_10_percent_data_model_checkpoint.weights.h5"
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                       save_weights_only=True,
                                                       monitor='val_accuracy',
                                                       save_best_only=True)

In [16]:
from tensorflow.keras import layers,Sequential

# from tensorflow.keras.models import Sequential

In [17]:
data_augmentation=Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomHeight(0.2),
    layers.RandomWidth(0.2)
],name='data_augmentation')   # As all the efficient net models built in deep learning have normalization built in so we can slip Rescaling(1/255.) required for Resnet-50 to be number between 0 and 1

In [18]:
data_augmentation

<Sequential name=data_augmentation, built=False>

In [19]:
base_model = tf.keras.applications.efficientnet.EfficientNetB0(include_top=False)
base_model.trainable=False

inputs=layers.Input(shape=(224,224,3),name='input_layer')
x=data_augmentation(inputs)
x=base_model(x,training=False)
x=layers.GlobalAveragePooling2D(name='global_average_pooling_layer')(x)
outputs=layers.Dense(len(train_data_all_10_percent.class_names),activation='softmax',name='output_layer')(x)
model=tf.keras.Model(inputs,outputs)


model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])


In [20]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ data_augmentation (Sequential)       │ (None, None, None, 3)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ efficientnetb0 (Functional)          │ (None, None, None, 1280)    │       4,049,571 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling_layer         │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output_layer (Dense)                 │ (None, 101)                 │         129,381 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,178,952 (15.94 MB)

 Trainable params: 129,381 (505.39 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [ ]:
history_all_classes_10_percent=model.fit(train_data_all_10_percent,epochs=5,
                                         validation_data=test_data,
                                         validation_steps=int(0.15*len(test_data)),
                                         callbacks=[checkpoint_callback])

In [21]:
#So now we have built our feature extraction transfer learning model
#Lets evaluate it on test_data
results_feature_extraction_model = model.evaluate(test_data)
results_feature_extraction_model


 10/790 ━━━━━━━━━━━━━━━━━━━━ 28:47 2s/step - accuracy: 0.0148 - loss: 4.4718

KeyboardInterrupt: 

In [ ]:
plot_loss_curves(history_all_classes_10_percent)

In [22]:
base_model.trainable=True
for layer in base_model.layers[:-5]:
  layer.trainable=False

In [28]:
model.layers

[<InputLayer name=input_layer, built=True>,
 <Sequential name=data_augmentation, built=True>,
 <Functional name=efficientnetb0, built=True>,
 <GlobalAveragePooling2D name=global_average_pooling_layer, built=True>,
 <Dense name=output_layer, built=True>]

In [29]:
for layer_number,layer in enumerate(model.layers[2].layers):
  print(layer_number,layer.name,layer.trainable)


0 input_layer_3 False
1 rescaling_4 False
2 normalization_2 False
3 rescaling_5 False
4 stem_conv_pad False
5 stem_conv False
6 stem_bn False
7 stem_activation False
8 block1a_dwconv False
9 block1a_bn False
10 block1a_activation False
11 block1a_se_squeeze False
12 block1a_se_reshape False
13 block1a_se_reduce False
14 block1a_se_expand False
15 block1a_se_excite False
16 block1a_project_conv False
17 block1a_project_bn False
18 block2a_expand_conv False
19 block2a_expand_bn False
20 block2a_expand_activation False
21 block2a_dwconv_pad False
22 block2a_dwconv False
23 block2a_bn False
24 block2a_activation False
25 block2a_se_squeeze False
26 block2a_se_reshape False
27 block2a_se_reduce False
28 block2a_se_expand False
29 block2a_se_excite False
30 block2a_project_conv False
31 block2a_project_bn False
32 block2b_expand_conv False
33 block2b_expand_bn False
34 block2b_expand_activation False
35 block2b_dwconv False
36 block2b_bn False
37 block2b_activation False
38 block2b_se_squeez

In [31]:
model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

In [32]:
for layer in model.layers:
  print(layer.name,layer.trainable)

input_layer True
data_augmentation True
efficientnetb0 True
global_average_pooling_layer True
output_layer True


In [33]:
for layer_number,layer in enumerate(base_model.layers):
  print(layer_number,layer.name,layer.trainable)

0 input_layer_3 False
1 rescaling_4 False
2 normalization_2 False
3 rescaling_5 False
4 stem_conv_pad False
5 stem_conv False
6 stem_bn False
7 stem_activation False
8 block1a_dwconv False
9 block1a_bn False
10 block1a_activation False
11 block1a_se_squeeze False
12 block1a_se_reshape False
13 block1a_se_reduce False
14 block1a_se_expand False
15 block1a_se_excite False
16 block1a_project_conv False
17 block1a_project_bn False
18 block2a_expand_conv False
19 block2a_expand_bn False
20 block2a_expand_activation False
21 block2a_dwconv_pad False
22 block2a_dwconv False
23 block2a_bn False
24 block2a_activation False
25 block2a_se_squeeze False
26 block2a_se_reshape False
27 block2a_se_reduce False
28 block2a_se_expand False
29 block2a_se_excite False
30 block2a_project_conv False
31 block2a_project_bn False
32 block2b_expand_conv False
33 block2b_expand_bn False
34 block2b_expand_activation False
35 block2b_dwconv False
36 block2b_bn False
37 block2b_activation False
38 block2b_se_squeez

In [34]:
# Fine-tune for 5 more epochs
fine_tune_epochs = 10 # model has already done 5 epochs, this is the total number of epochs we're after (5+5=10)

In [ ]:
history_all_classes_10_percent_fine_tune =model.fit(train_data_all_10_percent,
                                                    epochs=fine_tune_epochs,
                                                    initial_epoch=history_all_classes_10_percent[-1],
                                                    validation_data=test_data,
                                                    validation_steps=int(0.15*len(test_data)))

In [35]:
# Evaluate fine-tuned model on the whole test dataset
results_all_classes_10_percent_fine_tune = model.evaluate(test_data)
results_all_classes_10_percent_fine_tune


  4/790 ━━━━━━━━━━━━━━━━━━━━ 26:02 2s/step - accuracy: 0.0104 - loss: 4.4683

KeyboardInterrupt: 

In [ ]:
compare_historys(history_all_classes_10_percent,history_all_classes_10_percent_fine_tune,initial_epochs=5)

In [36]:
# To use the model in external we can save and load the model we will need to save and export it
model.save("drive/My Drive/tensorflow_course/101_food_class_10_percent_saved_big_dog_model")

ValueError: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `model.export(filepath)` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=drive/My Drive/tensorflow_course/101_food_class_10_percent_saved_big_dog_model.

In [ ]:
loaded_model=tf.keras.models.load_model("drive/My Drive/tensorflow_course/101_food_class_10_percent_saved_big_dog_model")

In [ ]:
loaded_model.evaluate(test_data)
model.evaluate(test_data)

In [1]:

import tensorflow as tf

# Download pre-trained model from Google Storage (like a cooking show, I trained this model earlier, so the results may be different than above)
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/06_101_food_class_10_percent_saved_big_dog_model.zip

--2024-09-02 07:40:41--  https://storage.googleapis.com/ztm_tf_course/food_vision/06_101_food_class_10_percent_saved_big_dog_model.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.97.207, 108.177.125.207, 142.250.157.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.97.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46760742 (45M) [application/zip]
Saving to: ‘06_101_food_class_10_percent_saved_big_dog_model.zip’

06_101_food_class_1 100%[===================>]  44.59M  12.0MB/s    in 3.7s    

2024-09-02 07:40:45 (12.0 MB/s) - ‘06_101_food_class_10_percent_saved_big_dog_model.zip’ saved [46760742/46760742]



In [20]:
saved_model_path='/content/06_101_food_class_10_percent_saved_big_dog_model'
# unzip_data(saved_model_path)

In [21]:
model = tf.keras.models.load_model(saved_model_path+"/saved_model.pb")

ValueError: File format not supported: filepath=/content/06_101_food_class_10_percent_saved_big_dog_model/saved_model.pb. Keras 3 only supports V3 `.keras` files and legacy H5 format files (`.h5` extension). Note that the legacy SavedModel format is not supported by `load_model()` in Keras 3. In order to reload a TensorFlow SavedModel as an inference-only layer in Keras 3, use `keras.layers.TFSMLayer(/content/06_101_food_class_10_percent_saved_big_dog_model/saved_model.pb, call_endpoint='serving_default')` (note that your `call_endpoint` might have a different name).

In [24]:
from tensorflow import keras

In [26]:
loaded_model=keras.layers.TFSMLayer('/content/06_101_food_class_10_percent_saved_big_dog_model', call_endpoint='serving_default')

In [28]:
pred_probs=model.predict(test_data,verbose=1)

AttributeError: 'TFSMLayer' object has no attribute 'evaluate'

In [ ]:
pred_probs.shape

In [ ]:
pred_probs[:10]

In [ ]:
sum(pred_probs[0]),len(pred_probs[0]),sum(pred_probs[0])

In [ ]:
print(f"Number of prediction probability for each invidual test data:{len(pred_probs[0])}")
print(f"Prediction probaility looks like: {pred_probs}")
print(f"Maximum value in the predicition probability array is {tf.argmax(pred_probs[0])}")

In [30]:
test_data.class_names[52]

'gyoza'

In [31]:
pred_classes=pred_probs.argmax(axis=0)
pred_classes[:10]

NameError: name 'pred_probs' is not defined

In [32]:
#Now we got the predictions array of all the predictions to evaluate them we need to compare it to original test data labels

In [33]:
test_data

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 101), dtype=tf.float32, name=None))>

In [34]:
import numpy as np
x=np.array([1,2,3,4])

In [38]:
y_labels=[]
for image,labels in test_data.unbatch():
  y_labels.append(labels.numpy().argmax())

print(y_labels[:10])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [39]:
len(y_labels)

25250

##Evaluating our model predictions
One way to check that our model predictions array is same as actual predictions array is by checking the accuracy score

In [40]:
from sklearn.metrics import accuracy_score
sklearn_accuracy=accuracy_score(y_labels,pred_classes)

NameError: name 'pred_classes' is not defined

In [ ]:
import numpy as np
np.isclose(results_downloaded_model[1],sklearn_accuracy)

In [41]:
class_names=test_data.class_names
class_names[:10]


['apple_pie',
 'baby_back_ribs',
 'baklava',
 'beef_carpaccio',
 'beef_tartare',
 'beet_salad',
 'beignets',
 'bibimbap',
 'bread_pudding',
 'breakfast_burrito']

In [46]:
import itertools
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix


make_confusion_matrix(y_labels,pred_classes,classes=class_names,figsize=(100,100),text_size=20,savefig=True)

NameError: name 'pred_classes' is not defined

In [47]:
from sklearn.metrics import classification_report

In [49]:
classification_report_dict=classification_report(y_labels,pred_classes,target_names=class_names,
                                                output_dict=True)

classification_report_dict

NameError: name 'pred_classes' is not defined

In [ ]:
class_f1_scores={}
for k,v in classification_report_dict.items():
  if k=='accuracy':
    break
  else:
    class_f1_scores[class_names[int(k)]]=v['f1_score']

print(class_f1_scores)

In [50]:
import pandas as pd
f1_scores=pd.DataFrame({'class_names':list(class_f1_scores.keys()),
                        'f1-score':list(class_f1_scores.values())}).sort_values('f1-score',ascending=False)


NameError: name 'class_f1_scores' is not defined

In [51]:
f1_scores[:10]

NameError: name 'f1_scores' is not defined

In [52]:
import matplotlib.pyplot as  plot
fig,ax=plt.subplots(figsize=(12,25))
scores=ax.barh(range(len(f1_scores),f1_scores['f1-score']))
ax.set_yticks(range(len(f1_scores)))
ax.set_yticklabels(list(f1_scores['class_names']))
ax.set_title("F1 classes for 10 different classes")
ax.set_xlabel("F1 Score")
ax.set_title("F1 score for each class")
ax.invert_yaxis();

def autolabel(rects):
  width=rects.get_width()
  ax.text(1.03*width,rect.get_y()+rect.get_height()/1.5,
          f"{width:.2f}",
          ha='center',va='bottom')

autolabel(scores)


NameError: name 'plt' is not defined

In [53]:
# Create a function to load and prepare images
def load_and_prep_image(filename,img_shape=224,scale=True):

  #Read in the image
  img=tf.io.read_file(filename)

  #Decode the image into tensor
  img=tf.io.decode_image(img,channels=3)

  #Resize the image
  img = tf.image.resize(img,size=[img_shape,img_shape])

  if scale:
    return img/255.
  else:
    return img

'101_food_classes_10_percent/test/'

In [ ]:
import random
import os

plt.figure(figsize=(17,10))
for i in range(3):
  class_name=random.choice(class_names)
  filename=os.list_dir(test_dir+'/'+class_name)
  filepath=test_dir+class_name+'/'+filename

  img=load_and_prep_image(filepath,scale=False)
  pred_prob=model.predict(tf.expand_dims(img,axis=0))
  pred_classes=class_names[pred_prob.argmax()]

  plt.subplot(1,3,i+1)
  plt.imshow(img/255.)
  if class_name=pred_classes:
    title_color='g'
  else:
    title_color='r'

  plt.title(f"Prediction is {pred_classes}, Acutal class_names is {class_name} and prediction probability is {pred_prob.max():.2f}".c=title_color)
  plt.axis(False)

In [60]:
filepaths=[]
for filepath in test_data.list_files("/content/101_food_classes_10_percent/test/*/*.jpg",shuffle=False):
  filepaths.append(filepath.numpy())

filepaths[:10]

[b'/content/101_food_classes_10_percent/test/apple_pie/1011328.jpg',
 b'/content/101_food_classes_10_percent/test/apple_pie/101251.jpg',
 b'/content/101_food_classes_10_percent/test/apple_pie/1034399.jpg',
 b'/content/101_food_classes_10_percent/test/apple_pie/103801.jpg',
 b'/content/101_food_classes_10_percent/test/apple_pie/1038694.jpg',
 b'/content/101_food_classes_10_percent/test/apple_pie/1047447.jpg',
 b'/content/101_food_classes_10_percent/test/apple_pie/1068632.jpg',
 b'/content/101_food_classes_10_percent/test/apple_pie/110043.jpg',
 b'/content/101_food_classes_10_percent/test/apple_pie/1106961.jpg',
 b'/content/101_food_classes_10_percent/test/apple_pie/1113017.jpg']

In [ ]:
import pandas as pd
pred_df=pd.Dataframe({"img_path":filepath,
              'y_true':y_labels,
              'y_pred':pred_classes,
              'pred_conf':pred_probs.max(axis=1),
              'y_true_classname':[class_names[i] for i in y_labels],
              'y_pred_classname':[class_names[i] for i in pred_classes]
              })

pred_df

In [ ]:
pred_df['pred_correct']=pred_df['y_true']==pred_df['y_pred']
pred_df

In [ ]:
top_100_wrong_predictions=pred_df[pred_df['pred_correct']==False].sort_values('pred_conf',ascending=False)[:100]
top_100_wrong_predictions

In [ ]:
image_to_view=9
start_index=0
plt.figure(figsize=(15,10))
for i,row in enumerate(top_100_wrong_predictions[start_index:start_index+image_to_view]).itertuples():
  plt.subplot(3,3,i+1):
  img=load_and_prep_image(row[1],scale=True)
  _,_,_,pred_prob,y_true,y_pred=row
  plt.imshow(img)
  plt.title(f"Actual: {y_true}, Prediction: {y_pred}, Probability: {pred_prob}")
  plt.axis(False)



In [61]:

# Download some custom images from Google Storage
# Note: you can upload your own custom images to Google Colab using the "upload" button in the Files tab
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/custom_food_images.zip

unzip_data("custom_food_images.zip")


--2024-09-02 11:22:56--  https://storage.googleapis.com/ztm_tf_course/food_vision/custom_food_images.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.204.207, 64.233.187.207, 64.233.188.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.204.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13192985 (13M) [application/zip]
Saving to: ‘custom_food_images.zip’

custom_food_images. 100%[===================>]  12.58M  8.82MB/s    in 1.4s    

2024-09-02 11:22:58 (8.82 MB/s) - ‘custom_food_images.zip’ saved [13192985/13192985]



In [63]:
import os

custom_food_images = ["custom_food_images/" + img_path for img_path in os.listdir("custom_food_images")]
custom_food_images

['custom_food_images/ramen.jpeg',
 'custom_food_images/sushi.jpeg',
 'custom_food_images/hamburger.jpeg',
 'custom_food_images/chicken_wings.jpeg',
 'custom_food_images/steak.jpeg',
 'custom_food_images/pizza-dad.jpeg']

In [ ]:
for img in custom_food_images:
  img=load_and_prep_image(img,scale=False)
  pred_prob=model.predict(tf.expand_dims(img,axis=0))
  pred_class=class_names[pred_prob.argmax()]
  plt.figure()
  plt.imshow(img/255.)
  plt.title(f'Prediction: {pred_class}, Pred Prob is {pred_prob.max():.2f}')
  plt.axis(False)

In [67]:
custom_images=['custom_food_images/' + img_pa for img_pa in os.listdir("custom_food_images")]
custom_images

['custom_food_images/ramen.jpeg',
 'custom_food_images/sushi.jpeg',
 'custom_food_images/hamburger.jpeg',
 'custom_food_images/chicken_wings.jpeg',
 'custom_food_images/steak.jpeg',
 'custom_food_images/pizza-dad.jpeg']